In [27]:
# Import dependencies
import pandas as pd
import requests
import time
import gmaps

# Import keys
from config import w_api_key
from config import gkey

In [28]:
# Load in and read City_Weather_data csv
weather_data = pd.read_csv("../WeatherPy/City_Weather_data.csv")
weather_data

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,khatanga,71.9667,102.5000,-13.00,85,0,8.28,RU,1612928439
1,vaini,-21.2000,-175.2000,78.80,94,75,17.27,TO,1612928441
2,albany,42.6001,-73.9662,21.99,86,2,7.05,US,1612928439
3,mataura,-46.1927,168.8643,50.67,91,100,11.30,NZ,1612928443
4,geraldton,-28.7667,114.6000,75.20,53,90,23.02,AU,1612928444
...,...,...,...,...,...,...,...,...,...
547,cortez,37.3489,-108.5859,46.40,36,90,6.91,US,1612929116
548,touros,-5.1989,-35.4608,80.60,78,40,6.91,BR,1612929118
549,cockburn town,21.4612,-71.1419,77.07,79,7,21.63,TC,1612928840
550,salym,60.0625,71.4789,-25.08,86,60,5.06,RU,1612929121


In [29]:
# Configure google heatmap
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Long"]].astype(float)

# Convert Poverty Rate to float and store
humidity = weather_data["Humidity"].astype(float)

In [31]:
# Create a humidity heatmap layer
fig = gmaps.figure(center=(0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=90,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# What are my preferred vacations conditions?
# 60-80 degrees
# Average cloudiness, 45-70??
# Humidty 50-65%

In [32]:
# Narrow down the dataframe to ideal conditions
ideal_weather = weather_data.loc[weather_data["Humidity"] >= 30]
ideal_weather = ideal_weather.loc[weather_data["Humidity"] <= 70]
ideal_weather = ideal_weather.loc[weather_data["Max Temp"] >= 55]
ideal_weather = ideal_weather.loc[weather_data["Max Temp"] <= 85]
ideal_weather = ideal_weather.loc[weather_data["Cloudiness"] >= 45]
ideal_weather = ideal_weather.loc[weather_data["Cloudiness"] <= 75]

ideal_weather["Nearby Hotel"] = ""

ideal_weather

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearby Hotel
19,port blair,11.6667,92.7500,81.32,56,47,13.44,IN,1612928463,
36,hilo,19.7297,-155.0900,73.40,64,75,17.27,US,1612928355,
69,salalah,17.0151,54.0924,75.20,35,75,10.36,OM,1612928527,
81,ponta do sol,32.6667,-17.1000,57.99,70,68,1.01,PT,1612928542,
115,wahiawa,21.5028,-158.0236,73.40,60,75,12.66,US,1612928584,
124,puerto carreno,6.1890,-67.4859,80.69,59,59,8.25,CO,1612928595,
193,kalmunai,7.4167,81.8167,78.48,70,51,11.79,LK,1612928676,
218,noumea,-22.2763,166.4572,82.40,69,75,12.66,NC,1612928451,
230,supaul,26.1167,86.6000,68.49,47,64,3.22,IN,1612928724,
248,nago,26.5881,127.9761,69.80,64,75,17.27,JP,1612928747,


In [33]:
# Run test to ensure gkey is properly configured from sample city (from first city pull)

waycross = "31.2136, -82.3540"
search_type = "hotel"
radius = 5000


params = {
    "location": waycross,
    "radius": radius,
    "types": search_type,
    "key": gkey
}

# "location": waycross as test case

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()
response["results"][1]["name"]
# Ok! Well, it is basically working.  So that's a good sign

'Super 8 by Wyndham Waycross GA'

In [34]:
# Find the first hotel within 5000 meters of each city

search_type = "hotel"
radius = 5000

params = {
    "radius": radius,
    "types": search_type,
    "key": gkey
}

for index, row in ideal_weather.iterrows():
    
    lat = row["Lat"]
    long = row["Long"]
    
    params["location"] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_returns = requests.get(base_url, params=params)
    
    hotel_returns = hotel_returns.json()
    
    print("Retrieving next hotel")
    
    try:
        ideal_weather.loc[index, "Nearby Hotel"] = hotel_returns["results"][1]["name"]
        
    except (KeyError, IndexError):
        print("No Hotel Available")
        ideal_weather.loc[index, "Nearby Hotel"] = np.nan
        pass
    
    time.sleep(1)
    

Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel


In [35]:
ideal_weather.dropna(inplace=True)
ideal_weather.reset_index(drop=True, inplace=True)
ideal_weather

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearby Hotel
0,port blair,11.6667,92.7500,81.32,56,47,13.44,IN,1612928463,Welcomhotel Bay Island Port Blair
1,hilo,19.7297,-155.0900,73.40,64,75,17.27,US,1612928355,Hilo Hawaiian Hotel
2,salalah,17.0151,54.0924,75.20,35,75,10.36,OM,1612928527,HAMDAN PLAZA HOTEL SALALAH
3,ponta do sol,32.6667,-17.1000,57.99,70,68,1.01,PT,1612928542,Hotel do Campo
4,wahiawa,21.5028,-158.0236,73.40,60,75,12.66,US,1612928584,Honolulu Jewelry Company
5,puerto carreno,6.1890,-67.4859,80.69,59,59,8.25,CO,1612928595,Reserva Natural Heliconia Amazonas
6,kalmunai,7.4167,81.8167,78.48,70,51,11.79,LK,1612928676,"Sri Subdraramaya, ශ්‍රී සුභද්‍රාරාමය කල්මුණේ ஸ..."
7,noumea,-22.2763,166.4572,82.40,69,75,12.66,NC,1612928451,Hôtel Le Lagon
8,supaul,26.1167,86.6000,68.49,47,64,3.22,IN,1612928724,State Bank of India
9,nago,26.5881,127.9761,69.80,64,75,17.27,JP,1612928747,Hotel Yugaf Inn Okinawa


In [36]:
locations_2 = ideal_weather[["Lat", "Long"]].astype(float)
humidity_2 = ideal_weather["Humidity"].astype(float)

info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearby Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]

fig_2 = gmaps.figure(center=(0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations_2, weights=humidity_2, 
                                 dissipating=False, max_intensity=70,
                                 point_radius = 1)

markers = gmaps.marker_layer(locations_2, info_box_content=hotel_info)

fig_2.add_layer(heat_layer)
fig_2.add_layer(markers)

fig_2

Figure(layout=FigureLayout(height='420px'))